In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [2]:
database1train = "CK_dataset/train"
database1test = "CK_dataset/test"
database2train = "JAFFE-[70,30]/train"
database2test = "JAFFE-[70,30]/test"

In [3]:
def load_and_detect_faces(folder_path):
    images = []
    labels = []
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face_region = img[y:y+h, x:x+w]
                        images.append(face_region)
                        labels.append(label)
    return images, labels

In [4]:
train_folder_path = database2train
test_folder_path = database2test
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [5]:
radius = 1
n_points = 8 * radius
def extract_lbp_features(images):
    lbp_features = []
    for img in images:
        img = cv2.resize(img, (64, 64))  # Standardize size
        lbp = local_binary_pattern(img, n_points, radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.append(hist)
    return np.array(lbp_features)

In [6]:
X_train_faces, y_train = load_and_detect_faces(train_folder_path)
X_train_features = extract_lbp_features(X_train_faces)
X_test_faces, y_test = load_and_detect_faces(test_folder_path)
X_test_features = extract_lbp_features(X_test_faces)

In [7]:
param_grid = {
    'svc__C': [4, 4.5, 5, 6, 7, 8, 9, 10],  
    'svc__gamma': ['scale'], 
    'svc__kernel': ['rbf']
svm_pipeline = make_pipeline(StandardScaler(), SVC())
grid_search = GridSearchCV(svm_pipeline, param_grid, refit=True, verbose=3, cv=10)
grid_search.fit(X_train_features, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)



SyntaxError: '{' was never closed (1967556147.py, line 1)

In [ ]:
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=4.5, gamma='scale'))
svm_classifier.fit(X_train_features, y_train)
y_pred = svm_classifier.predict(X_test_features)
svm_classifier.fit(X_train_features, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
y_pred = svm_classifier.predict(X_test_features)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)
print(len(y_test))
print(conf_matrix)

In [ ]:
class_names = ["Angry", "Fear", "Happy", "Neutral", "Sadness", "Surprise"]
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_percent = conf_matrix / conf_matrix.sum(axis=1, keepdims=True) * 100
plt.figure(figsize=(8, 8))
ax = sns.heatmap(conf_matrix, annot=False, cmap="BuGn", xticklabels=class_names, yticklabels=class_names, cbar=False)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        count = conf_matrix[i, j]
        percent = conf_matrix_percent[i, j]
        ax.text(j + 0.5, i + 0.5, f"{count}\n({percent:.1f}%)", ha="center", va="center", color="black", fontsize=10)

plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("SVM Database2")
plt.show()


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k_range = range(1, 25)
best_knn_accuracy = 0
best_k = 1

for k in k_range:
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train_features, y_train)
    y_pred = knn_model.predict(X_test_features)
    knn_accuracy = accuracy_score(y_test, y_pred)
    if knn_accuracy > best_knn_accuracy:
        best_knn_accuracy = knn_accuracy
        best_k = k
final_knn_model = KNeighborsClassifier(n_neighbors=best_k)
final_knn_model.fit(X_train_features, y_train)
y_pred = final_knn_model.predict(X_test_features)


In [ ]:
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("Best k:", best_k)
print("Accuracy:", knn_accuracy)
print(classification_report(y_test, knn_predictions))

In [ ]:
knn_predictions = final_knn_model.predict(X_test_features)
conf_matrix_knn = confusion_matrix(y_test, knn_predictions)
class_names = ["Angry", "Fear", "Happy", "Neutral", "Sadness", "Surprise"]
conf_matrix_percent_knn = conf_matrix_knn / conf_matrix_knn.sum(axis=1, keepdims=True) * 100
plt.figure(figsize=(8, 8))
ax = sns.heatmap(conf_matrix_knn, annot=False, cmap="BuGn", xticklabels=class_names, yticklabels=class_names, cbar=False)
for i in range(conf_matrix_knn.shape[0]):
    for j in range(conf_matrix_knn.shape[1]):
        count = conf_matrix_knn[i, j]
        percent = conf_matrix_percent_knn[i, j]
        ax.text(j + 0.5, i + 0.5, f"{count}\n({percent:.1f}%)", ha="center", va="center", color="black", fontsize=10)
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("KNN Database2")
plt.show()
